In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [3]:
# Settings
TEST_BATCH_SIZE = 128
NUM_WORKERS = 4

# Dataset hyper-parameters
WINDOW_SIZE_MS=100
STRIDE_MS=50

In [4]:
import torch
from torch.utils.data import DataLoader
from src import LazyWindowedDataset

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=WINDOW_SIZE_MS,
    stride_ms=STRIDE_MS,
)
test_loader = DataLoader(test_source_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)

In [5]:
from models import RoPeTimeSeriesTransformer
from src import evaluate, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, _, _, _, _, _, _ = load_model_checkpoint("checkpoints/bm_aerodactyl_RoPeTimeSeriesTransformer_epoch_1.pt", RoPeTimeSeriesTransformer, optimizer_class=None)
model.to(device)
criterion = torch.nn.MSELoss()

In [6]:
loss, auc = evaluate(model, test_loader, criterion, merge_strategy='stack', verbose=1)
print(f"Overall S+T | Loss: {loss:.4f}, AUC: {auc:.4f}")

Evaluation: 100%|██████████| 251/251 [02:00<00:00,  2.08batch/s]

Overall S+T | Loss: 0.7745, AUC: 0.4642


In [7]:
acc_loss, acc_auc = evaluate(model, test_loader, criterion, sensors_to_test=['acc'], merge_strategy='stack', verbose=1)
print(f"Acc s+T | Loss: {acc_loss:.4f}, AUC: {acc_auc:.4f}")

Evaluation: 100%|██████████| 251/251 [02:04<00:00,  2.01batch/s]

Acc s+T | Loss: 0.3816, AUC: 0.4726


In [8]:
gyro_loss, gyro_auc = evaluate(model, test_loader, criterion, sensors_to_test=['gyro'], merge_strategy='stack', verbose=1)
print(f"Gyro S+T | Loss: {gyro_loss:.4f}, AUC: {gyro_auc:.4f}")

Evaluation: 100%|██████████| 251/251 [01:54<00:00,  2.19batch/s]

Gyro S+T | Loss: 0.3699, AUC: 0.5046


In [9]:
mic_loss, mic_auc = evaluate(model, test_loader, criterion, sensors_to_test=['mic'], merge_strategy='stack', verbose=1)
print(f"Mic S+T | Loss: {mic_loss:.4f}, AUC: {mic_auc:.4f}")

Evaluation: 100%|██████████| 251/251 [01:52<00:00,  2.24batch/s]

Mic S+T | Loss: 0.0611, AUC: 0.5369
